<a href="https://colab.research.google.com/github/yuyu990116/CS224W-assignment/blob/main/P3_peft_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

Mounted at /content/drive


In [ ]:
!pip install peft==0.5.0
!pip install accelerate==0.22.0
!pip install transformers==4.33.1
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

In [ ]:
#lora适配自定义模型

In [ ]:
n1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
n1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
for name,param in n1.named_parameters():
  print(name,param)

0.weight Parameter containing:
tensor([[ 2.5363e-01, -8.5284e-02, -2.4243e-01, -1.0562e-01, -3.0322e-01,
         -3.1527e-01, -1.8856e-02,  1.4177e-01, -2.4148e-01,  2.5017e-01],
        [-5.8837e-02,  1.4258e-02,  1.0422e-01,  1.1408e-01,  1.7103e-04,
         -2.4463e-01, -2.0892e-01,  7.3016e-02,  4.7191e-02, -8.7792e-02],
        [-8.1819e-02,  2.4983e-01,  1.4444e-02,  1.8578e-01, -2.3290e-01,
         -8.3650e-02,  9.1456e-02,  2.9422e-02, -4.2514e-02,  1.6925e-01],
        [-2.4790e-01,  3.9774e-03, -6.3456e-02,  3.0024e-01, -2.1813e-01,
          1.5836e-01,  4.8530e-02,  1.1130e-01, -1.3256e-02,  8.6003e-02],
        [ 1.4101e-01,  6.7990e-02,  3.0572e-01,  1.6322e-01,  3.0823e-01,
          8.4492e-02,  7.3510e-02,  2.9989e-02, -1.7748e-01, -1.0538e-01],
        [ 2.8151e-01, -1.0977e-01, -2.4836e-01, -2.4062e-01, -1.2866e-01,
          6.4993e-02,  5.4429e-02, -7.4712e-02, -2.1652e-01,  2.6645e-01],
        [-1.5677e-01,  1.9531e-01,  7.3288e-02,  2.0439e-01, -8.5654e-02,
 

In [ ]:
config = LoraConfig(target_modules=["0"]) #必须指定是哪一层
model1 = get_peft_model(n1,config)
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [ ]:
for name,param in model1.named_parameters():
  print(name,param)

base_model.model.0.weight Parameter containing:
tensor([[ 0.1711, -0.1354,  0.1002, -0.2820, -0.1607, -0.2068, -0.2129,  0.0887,
          0.0664,  0.1450],
        [-0.1965, -0.2049, -0.2242,  0.0610, -0.2919,  0.1789,  0.2719,  0.1356,
         -0.1253,  0.1001],
        [ 0.0414,  0.0382, -0.2861, -0.1712,  0.0269,  0.0650, -0.1832,  0.1839,
          0.0099, -0.0717],
        [ 0.2443,  0.0159,  0.1061,  0.2934,  0.0914,  0.1048, -0.2877, -0.0402,
          0.0090, -0.1782],
        [ 0.0295,  0.0974, -0.1635, -0.2743, -0.0330, -0.1744, -0.2096,  0.0075,
         -0.1573, -0.2911],
        [ 0.1987,  0.0814,  0.0769,  0.0711, -0.0115,  0.0509,  0.2471, -0.0517,
         -0.2011,  0.0488],
        [-0.2833, -0.1283, -0.0465, -0.1109,  0.0841,  0.0360, -0.1508, -0.2019,
          0.1578,  0.2165],
        [ 0.1703,  0.1016,  0.1869,  0.1596,  0.1274, -0.0516,  0.0561, -0.1447,
          0.2953,  0.0043],
        [ 0.1886, -0.2352,  0.2507, -0.2805,  0.2728,  0.1055, -0.2156, -0.2070,

In [ ]:
#一个主模型，多个适配器

In [ ]:
n1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)

config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(n1, config1)
model2.save_pretrained("./loraA")

n1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
config2 = LoraConfig(target_modules=["2"])
model3 = get_peft_model(n1, config2)
model3.save_pretrained("./loraB")

In [ ]:
n1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
model4 = PeftModel.from_pretrained(n1, model_id="./loraA/", adapter_name="loraA")
model4

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [ ]:
model4.load_adapter("./loraB/", adapter_name="loraB")
model4

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(
        in_features=10, out_features=2, bias=True
        (lora_dropout): ModuleDict(
          (loraB): Identity()
        )
        (lora_A): ModuleDict(
          (loraB): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraB): Linear(in_features=8, out_features=2, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
     

In [ ]:
model4.active_adapter #查看现在是哪个LOra在起作用

'loraB'

In [ ]:
model4.set_adapter("loraB") #切换lora

In [ ]:
#禁用适配器
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))

tensor([[0.7081, 0.8522]])
